The sys.setrecursionlimit function is used to increase the recursion limit, which helps prevent potential recursion errors when running complex models with deep nested functions or when using certain libraries like TensorFlow.

In [1]:
import sys

# Increase recursion limit to prevent potential issues
sys.setrecursionlimit(100000)

In [2]:
# Import necessary libraries
import os
import warnings

import keras_tuner as kt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Suppress all Python warnings
warnings.filterwarnings('ignore')

# Set TensorFlow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = all logs, 1 = filter out INFO, 2 = filter out INFO and WARNING, 3 = ERROR only

In [3]:
# Load and preprocess the MNIST dataset
(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train, x_val = x_train / 255.0, x_val / 255.0

print(f'Training data shape: {x_train.shape}')
print(f'Validation data shape: {x_val.shape}')

Training data shape: (60000, 28, 28)
Validation data shape: (10000, 28, 28)


**Define Model with Hyperparameters**

In [4]:
def build_model(hp):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
        Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

**Configure Hyperparameter Search**

In [5]:
# Create a RandomSearch Tuner 
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Display a summary of the search space 
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


**Run Hyperparameter Search**

In [6]:
# Run the hyperparameter search 
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val)) 

# Display a summary of the results 
tuner.results_summary() 

Trial 10 Complete [00h 01m 05s]
val_accuracy: 0.969650000333786

Best val_accuracy So Far: 0.9801499843597412
Total elapsed time: 00h 08m 34s
Results summary
Results in my_dir\intro_to_kt
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 07 summary
Hyperparameters:
units: 416
learning_rate: 0.0009124105694925094
Score: 0.9801499843597412

Trial 00 summary
Hyperparameters:
units: 352
learning_rate: 0.00178701749121327
Score: 0.9783500134944916

Trial 03 summary
Hyperparameters:
units: 352
learning_rate: 0.00034109005522774783
Score: 0.9780499935150146

Trial 04 summary
Hyperparameters:
units: 160
learning_rate: 0.0032285872305509375
Score: 0.9760000109672546

Trial 06 summary
Hyperparameters:
units: 480
learning_rate: 0.005548950103091091
Score: 0.972350001335144

Trial 05 summary
Hyperparameters:
units: 128
learning_rate: 0.0003492037347843834
Score: 0.9720499813556671

Trial 09 summary
Hyperparameters:
units: 384
learning_rate: 0.005440954850111082
Score: 0

**Analyze and Use Best Hyperparameters**

In [7]:
# Retrieve the best hyperparameters 
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0] 
print(f""" 
The optimal number of units in the first dense layer is {best_hps.get('units')}. 
The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}. 
""") 

# Build and Train the Model with Best Hyperparameters 
model = tuner.hypermodel.build(best_hps) 
model.fit(x_train, y_train, epochs=10, validation_split=0.2) 

# Evaluate the model on the test set 
test_loss, test_acc = model.evaluate(x_val, y_val) 
print(f'Test accuracy: {test_acc}') 

 
The optimal number of units in the first dense layer is 416. 
The optimal learning rate for the optimizer is 0.0009124105694925094. 

Epoch 1/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2376 - accuracy: 0.9311 - val_loss: 0.1170 - val_accuracy: 0.9653
Epoch 2/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0950 - accuracy: 0.9714 - val_loss: 0.0973 - val_accuracy: 0.9688
Epoch 3/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0606 - accuracy: 0.9817 - val_loss: 0.0884 - val_accuracy: 0.9742
Epoch 4/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0418 - accuracy: 0.9866 - val_loss: 0.0837 - val_accuracy: 0.9749
Epoch 5/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0288 - accuracy: 0.9911 - val_loss: 0.0833 - val_accuracy: 0.9762
Epoch 6/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0246 - accuracy: 0.9926 - val_loss: 0.0887 - val_accuracy: 0.9758


## Practice Exercise

**Set Keras Tuner**

In [8]:
# Load and preprocess the MNIST data set 
(x_train, y_train), (x_val, y_val) = mnist.load_data() 
x_train, x_val = x_train / 255.0, x_val / 255.0 

# Print the shapes of the training and validation datasets
print(f'Training data shape: {x_train.shape}') 
print(f'Validation data shape: {x_val.shape}')

Training data shape: (60000, 28, 28)
Validation data shape: (10000, 28, 28)


**Define Model with Hyperparameters**

In [9]:
# Define a model-building function
def build_model(hp):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
        Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

**Configure Hyperparameter Search**

In [10]:
# Create a RandomSearch Tuner
tuner = kt.RandomSearch(
    build_model,  # Ensure 'build_model' function is defined from previous code
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Display a summary of the search space
tuner.search_space_summary()

Reloading Tuner from my_dir\intro_to_kt\tuner0.json
Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


**Run Hyperparameter Search**

In [11]:
# Run the hyperparameter search 
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val)) 

 # Display a summary of the results 
tuner.results_summary()

Results summary
Results in my_dir\intro_to_kt
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 07 summary
Hyperparameters:
units: 416
learning_rate: 0.0009124105694925094
Score: 0.9801499843597412

Trial 00 summary
Hyperparameters:
units: 352
learning_rate: 0.00178701749121327
Score: 0.9783500134944916

Trial 03 summary
Hyperparameters:
units: 352
learning_rate: 0.00034109005522774783
Score: 0.9780499935150146

Trial 04 summary
Hyperparameters:
units: 160
learning_rate: 0.0032285872305509375
Score: 0.9760000109672546

Trial 06 summary
Hyperparameters:
units: 480
learning_rate: 0.005548950103091091
Score: 0.972350001335144

Trial 05 summary
Hyperparameters:
units: 128
learning_rate: 0.0003492037347843834
Score: 0.9720499813556671

Trial 09 summary
Hyperparameters:
units: 384
learning_rate: 0.005440954850111082
Score: 0.969650000333786

Trial 01 summary
Hyperparameters:
units: 384
learning_rate: 0.0001079063777579721
Score: 0.9658499956130981

Trial 02 summar

**Analyze and Use Best Hyperparameter**

In [12]:
# Retrieve the best hyperparameters 
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0] 
print(f""" 
The optimal number of units in the first dense layer is {best_hps.get('units')}. 
The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}. 
""") 

# Build and train the model with best hyperparameters 
model = tuner.hypermodel.build(best_hps) 
model.fit(x_train, y_train, epochs=10, validation_split=0.2) 

# Evaluate the model on the validation set 
val_loss, val_acc = model.evaluate(x_val, y_val) 
print(f'Validation accuracy: {val_acc}') 

 
The optimal number of units in the first dense layer is 416. 
The optimal learning rate for the optimizer is 0.0009124105694925094. 

Epoch 1/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2376 - accuracy: 0.9307 - val_loss: 0.1347 - val_accuracy: 0.9603
Epoch 2/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0960 - accuracy: 0.9710 - val_loss: 0.1106 - val_accuracy: 0.9647
Epoch 3/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0627 - accuracy: 0.9808 - val_loss: 0.0859 - val_accuracy: 0.9735
Epoch 4/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0420 - accuracy: 0.9871 - val_loss: 0.0831 - val_accuracy: 0.9762
Epoch 5/10
1500/1500 [==============================] - 5s 4ms/step - loss: 0.0318 - accuracy: 0.9903 - val_loss: 0.0780 - val_accuracy: 0.9765
Epoch 6/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.0223 - accuracy: 0.9928 - val_loss: 0.0832 - val_accuracy: 0.9787
